In [1]:
import os
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from numpy import sin,cos,arcsin,sqrt,abs,pi,log10,exp
from scipy.fftpack import fft,ifft

import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
%matplotlib inline

#obspy
from obspy import UTCDateTime
from obspy import read
from obspy import read_inventory

from obspy.taup import TauPyModel
model = TauPyModel(model="iasp91")

from obspy.core.stream import Stream
from obspy.core.event import read_events
from obspy.core.inventory.inventory import read_inventory
from obspy.core.util import Enum

from obspy.core.event.magnitude import StationMagnitude

from obspy.clients.neic import Client as nClient
from obspy.clients.fdsn import Client as fdsnClient

from obspy.geodetics.base import locations2degrees as l2d
from obspy.geodetics.base import degrees2kilometers as d2km
from obspy.geodetics.base import kilometers2degrees as km2d
from obspy.geodetics.base import gps2dist_azimuth as ll2az

from obspy.core.inventory.response import ResponseListElement as amplitude

from obspy.signal.filter import bandpass

#rtergpy
from rtergpy.run import defaults, event, etime2name, src2ergs
from rtergpy.waveforms import get_respinv, getwaves, loadwaves, process_waves, trstat2pd

from locale import setlocale
from scipy.fftpack import fft,ifft
from scipy.stats import gmean 
from tqdm import tqdm
from compress_pickle import dump as cpkldump # reading/writing compressed pickles
from compress_pickle import load as cpklload # reading/writing compressed pickles

In [2]:
Defaults=defaults()
Event=event()

Defaults.src ="RASPISHAKE"
src = Defaults.src 

Defaults.network ="AM"
network = Defaults.network

Defaults.chan ="EHZ"
chan = Defaults.chan

Defaults.stationrange=[1.,10.]
rads = Defaults.stationrange

Event.eloc = [18.4578,-73.3389,10]
eloc = Event.eloc

Event.etime = UTCDateTime(2022,1,24,13,16,23.425)
etime = Event.etime

Event.origin =[eloc,etime]
Event.ecount ='00'
Event.iter ='RS'
Event.eventname = etime2name(etime,ecount=Event.ecount)

# getting inventory

inventory = get_respinv(network,eloc,etime,rads,chan,src)
print(inventory)

Inventory created at 2023-06-16T20:07:39.152554Z
	Sending institution: SeisComP (RaspberryShake)
	Contains:
		Networks (1):
			AM
		Stations (28):
			AM.R0306 (Raspberry Shake Citizen Science Station)
			AM.R095E (Raspberry Shake Citizen Science Station)
			AM.R17FC (Raspberry Shake Citizen Science Station)
			AM.R2974 (Raspberry Shake Citizen Science Station)
			AM.R2C45 (Raspberry Shake Citizen Science Station)
			AM.R34A9 (Raspberry Shake Citizen Science Station)
			AM.R4802 (Raspberry Shake Citizen Science Station)
			AM.R4DB9 (Raspberry Shake Citizen Science Station)
			AM.R4EEE (Raspberry Shake Citizen Science Station)
			AM.R54B1 (Raspberry Shake Citizen Science Station)
			AM.R6324 (Raspberry Shake Citizen Science Station)
			AM.R804D (Raspberry Shake Citizen Science Station)
			AM.R976A (Raspberry Shake Citizen Science Station)
			AM.RA08A (Raspberry Shake Citizen Science Station)
			AM.RA887 (Raspberry Shake Citizen Science Station)
			AM.RA9CF (Raspberry Shake Citizen Scienc

In [3]:
# getting waveforms
Event.newData = True  # False means locally stored

if Event.newData:
    print("Getting waveforms")
    #st,df=getwaves(eloc,etime,pwindow=Def.waveparams[1],rads=Def.stationrange)
    st,df=getwaves(Defaults=Defaults,Event=Event)
else:
    print("Loading locally stored waveforms")
    try:
        st,df=loadwaves(Defaults=Defaults,Event=Event) 
    except:
        print("Couldn't load data for "+eventname+". Attempting to download:")
        st,df=getwaves(Defaults=Defaults,Event=Event)
if len(st) == 0:
    raise Exception("ERROR: No waveforms retreived.") 

Getting waveforms
New working directory is:  /Users/hkunwer/Documents/research/EQenergy/events/2022/2022012400/RS
Checking for stations available within range from IRIS
Getting waves from RASPISHAKE


  0%|                                                    | 0/28 [00:00<?, ?it/s]

10 18.4578 -73.3389 18.52252252 -72.24929548


  4%|█▌                                          | 1/28 [00:01<00:52,  1.96s/it]

10 18.4578 -73.3389 25.92792793 -80.41194326


...

 96%|█████████████████████████████████████████▍ | 27/28 [00:43<00:01,  1.50s/it]

10 18.4578 -73.3389 18.08108108 -67.04209886


100%|███████████████████████████████████████████| 28/28 [00:43<00:00,  1.57s/it]

AM.S897D.00.EHZ failed to download. Continuing..
rPtime =  60.0
Trace SNR =  33507.00153670628 1
Trace SNR =  1.9535977173105086 2
Trace SNR =  1.1190312407952294 3


Trace SNR =  480.8589546770031 4
Trace SNR =  1.4192637527994663 5
Trace SNR =  82.47129498163677 6
Trace SNR =  1.29898385468013 7
Trace SNR =  191.81053272544594 8
Trace SNR =  1.3713020620842287 9
Trace SNR =  0.9804724314212754 10
Trace SNR =  13951.268120281064 11
Trace SNR =  175.1629656881115 12
Trace SNR =  1.427339646787087 13
Trace SNR =  1.2224276212058625 14
Trace SNR =  1.4254295476506453 15
Trace SNR =  3.306360285749013 16
Trace SNR =  176.94027532811185 17
Trace SNR =  166888.31896330457 18
Trace SNR =  6.006171446784444 19
Trace SNR =  0.9928295704720193 20
Trace SNR =  416.22344514400004 21
Trace SNR =  2.625076714997328 22
9  of  22 traces were kept above SNR threshold 10.0
writing  /Users/hkunwer/Documents/research/EQenergy/events/2022/2022012400/RS/pkls/Params_2022012400.pkl 
 /Users/hkunwer/Documents/research/EQenergy/events/2022/2022012400/RS/Params_2022012400.csv
writing  /Users/hkunwer/Documents/research/EQenergy/events/2022/2022012400/RS/pkls/Wavestream-raw_20

In [4]:
# filter for instrument response and taper

taper=0.05

stp=st.copy()  # create backup
# process data
stp.detrend(type='polynomial', order=5) # pre-instrument removal
stp.taper(taper)
stp.remove_response(output="DISP")
stp.detrend(type='polynomial', order=5) # post-instrument removal
stp.taper(taper)

9 Trace(s) in Stream:
AM.R0306.00.EHZ | 2022-01-24T13:15:43.425999Z - 2022-01-24T13:21:43.425999Z | 100.0 Hz, 36001 samples
AM.R2974.00.EHZ | 2022-01-24T13:16:51.211999Z - 2022-01-24T13:22:51.211999Z | 100.0 Hz, 36001 samples
AM.R34A9.00.EHZ | 2022-01-24T13:17:00.989999Z - 2022-01-24T13:23:00.989999Z | 100.0 Hz, 36001 samples
AM.R4DB9.00.EHZ | 2022-01-24T13:16:54.835000Z - 2022-01-24T13:22:54.835000Z | 100.0 Hz, 36001 samples
AM.RA08A.00.EHZ | 2022-01-24T13:16:08.553000Z - 2022-01-24T13:22:08.553000Z | 100.0 Hz, 36001 samples
AM.RA9CF.00.EHZ | 2022-01-24T13:17:02.590000Z - 2022-01-24T13:23:02.590000Z | 100.0 Hz, 36001 samples
AM.RD17E.00.EHZ | 2022-01-24T13:16:52.256999Z - 2022-01-24T13:22:52.256999Z | 100.0 Hz, 36001 samples
AM.RD4D8.00.EHZ | 2022-01-24T13:16:06.957000Z - 2022-01-24T13:21:59.197000Z | 100.0 Hz, 35225 samples
AM.RE001.00.EHZ | 2022-01-24T13:16:27.183000Z - 2022-01-24T13:22:27.183000Z | 100.0 Hz, 36001 samples

In [5]:
# process each individual frequency band in a bandpass using .filter
# frequency bands of 0.1-0.25, 0.25-0.5, 0.5-0.75, 0.75-1, 1-1.5

freqs = ([0.1,0.25],[0.25,0.5],[0.50,0.75],[0.75,1],[1,1.5])

# freq1 band = 0.1-0.25,
stp_freq1 = stp.copy()
stp_freq1.filter("bandpass",freqmin=0.10 , freqmax=0.25)

# freq2 band = 0.25-0.5,
stp_freq2 = stp.copy()
stp_freq2.filter("bandpass", freqmin=0.25, freqmax=0.50)

# freq3 band = 0.5-0.75,
stp_freq3 = stp.copy()
stp_freq3.filter("bandpass", freqmin=0.50, freqmax=0.75)

# freq4 band = 0.75-1.
stp_freq4 = stp.copy()
stp_freq4.filter("bandpass", freqmin=0.75, freqmax=1.0)

# freq5 band = 0.75-1.
stp_freq5 = stp.copy()
stp_freq5.filter("bandpass", freqmin=1, freqmax=1.5)

#add all freq to a tuple
stp_freqs = (stp_freq1,stp_freq2,stp_freq3,stp_freq4,stp_freq5)

In [6]:
# unsure if need this in final code, if not how would i call it? 
# Maybe make a functions script and call from that??? Get on this hiba!!!

def freqmaxes(trace, freqs):
    
    data = trace.data                           # the actual data
    samp = data.shape[-1]                       # number of sample point
    delta = trace.stats.delta                  # sample spacing 
    time = trace.times()                       # time axis 

    # ---- compute fft for our data
    fftdata = np.fft.rfft(data, n=samp)

    # ---- Fdat has real and complex part, to compute power spectrum we take the
    fftdata_abs = np.abs(fftdata)

    # ---- compute x-axis in frequency-domain for plotting
    xf = np.linspace(0.0, 1.0/(2.0*delta), int(samp/2)+1)

    maxamps = []
    idx_of_max_amps = []
    
    for freq in freqs: 
        #argmin find specific position here
        idx_of_min_freq = (np.abs(xf-freq[0])).argmin()
        idx_of_max_freq = (np.abs(xf-freq[1])).argmin()
        
        fftdata_abs_slice = fftdata_abs[idx_of_min_freq:idx_of_max_freq]
        
        # ---- get the max value of amplitude spectrum
        maxamp = fftdata_abs_slice.max()
        idx_of_max_amp = int(np.where(fftdata_abs_slice == maxamp) + idx_of_min_freq)
        maxamps.append(1.0/samp * maxamp)
        idx_of_max_amps.append(idx_of_max_amp)
    
    return maxamps, idx_of_max_amps, xf

In [7]:
# use freqmaxes function to get maxamps for all traces in one frequency band
# compare the plots formed from the function above with previous tests in Appendix below

ALLmaxamps, ALLidx_of_max_amps, trname = [], [], []

for st in stp_freqs:

    for tr in st:
        
        # print(f"Now processing trace {tr}")
        maxamps, idx_of_max_amps, xf = freqmaxes(tr, freqs)
        # print(maxamps, idx_of_max_amps) # to test if it worked
        ALLmaxamps.append(maxamps)
        ALLidx_of_max_amps.append(idx_of_max_amps)
        trname.append(tr)